In [1]:
# !pip install hierarchicalforecast
# !pip install -U statsforecast numba

In [ ]:
# !pip install quadprog

In [ ]:
# !pip install hierarchicalforecast

In [1]:
import numpy as np
import pandas as pd

Load up the data file

In [87]:
Y_df = pd.read_csv('C:\\Users\\K05048\\Desktop\\IBF\\LIFESalesJanuary1998_October2017.csv')
Y_df.head()

,State,Region,Product,ds,y
0,Louisiana,Acadiana,Term Life,1998-01-01,135.08
1,Louisiana,Acadiana,Term Life,1998-04-01,109.99
2,Louisiana,Acadiana,Term Life,1998-07-01,166.03
3,Louisiana,Acadiana,Term Life,1998-10-01,127.16
4,Louisiana,Acadiana,Term Life,1999-01-01,137.45


Define the Hierarchical structure by the columns

In [88]:
hierarchy_spec = [
    ['State'],
    ['State', 'Region'],
    ['State', 'Region', 'Product']
]

Use the aggregate function to set up the tags, and Summing table for computation

In [89]:
from hierarchicalforecast.utils import aggregate

In [90]:
Y_df, S_df, tags = aggregate(Y_df, hierarchy_spec)
Y_df = Y_df.reset_index()

The Unique_id field holds the tags, which are the combination of the State, Region and Product fields.

In [91]:
Y_df.head()

,unique_id,ds,y
0,Louisiana,1998-01-01,23182.23
1,Louisiana,1998-04-01,20323.38
2,Louisiana,1998-07-01,19826.70
3,Louisiana,1998-10-01,20830.13
4,Louisiana,1999-01-01,22087.48


In [93]:
Y_df.unique_id.unique().tolist()

['Louisiana',
 'Louisiana/Acadiana',
 'Louisiana/Bayou',
 'Louisiana/Central Louisiana',
 'Louisiana/Florida Parishes',
 'Louisiana/Greater New Orleans',
 'Louisiana/North Lake',
 'Louisiana/North Louisiana',
 'Louisiana/Southwest Louisiana',
 'Louisiana/Acadiana/Annuities',
 'Louisiana/Acadiana/Term Life',
 'Louisiana/Acadiana/Whole Life',
 'Louisiana/Bayou/Annuities',
 'Louisiana/Bayou/Term Life',
 'Louisiana/Bayou/Whole Life',
 'Louisiana/Central Louisiana/Annuities',
 'Louisiana/Central Louisiana/Term Life',
 'Louisiana/Central Louisiana/Whole Life',
 'Louisiana/Florida Parishes/Annuities',
 'Louisiana/Florida Parishes/Term Life',
 'Louisiana/Florida Parishes/Whole Life',
 'Louisiana/Greater New Orleans/Annuities',
 'Louisiana/Greater New Orleans/Term Life',
 'Louisiana/Greater New Orleans/Whole Life',
 'Louisiana/North Lake/Annuities',
 'Louisiana/North Lake/Term Life',
 'Louisiana/North Lake/Whole Life',
 'Louisiana/North Louisiana/Annuities',
 'Louisiana/North Louisiana/Term Lif

Summing table below

In [94]:
S_df.iloc[:6, :6]

,Louisiana/Acadiana/Annuities,Louisiana/Acadiana/Term Life,Louisiana/Acadiana/Whole Life,Louisiana/Bayou/Annuities,Louisiana/Bayou/Term Life,Louisiana/Bayou/Whole Life
Louisiana,1.0,1.0,1.0,1.0,1.0,1.0
Louisiana/Acadiana,1.0,1.0,1.0,0.0,0.0,0.0
Louisiana/Bayou,0.0,0.0,0.0,1.0,1.0,1.0
Louisiana/Central Louisiana,0.0,0.0,0.0,0.0,0.0,0.0
Louisiana/Florida Parishes,0.0,0.0,0.0,0.0,0.0,0.0
Louisiana/Greater New Orleans,0.0,0.0,0.0,0.0,0.0,0.0


Tags structure will be used for reconcilliation

In [8]:
tags['State/Region/Product']

array(['Louisiana/Acadiana/Annuities', 'Louisiana/Acadiana/Term Life',
       'Louisiana/Acadiana/Whole Life', 'Louisiana/Bayou/Annuities',
       'Louisiana/Bayou/Term Life', 'Louisiana/Bayou/Whole Life',
       'Louisiana/Central Louisiana/Annuities',
       'Louisiana/Central Louisiana/Term Life',
       'Louisiana/Central Louisiana/Whole Life',
       'Louisiana/Florida Parishes/Annuities',
       'Louisiana/Florida Parishes/Term Life',
       'Louisiana/Florida Parishes/Whole Life',
       'Louisiana/Greater New Orleans/Annuities',
       'Louisiana/Greater New Orleans/Term Life',
       'Louisiana/Greater New Orleans/Whole Life',
       'Louisiana/North Lake/Annuities', 'Louisiana/North Lake/Term Life',
       'Louisiana/North Lake/Whole Life',
       'Louisiana/North Louisiana/Annuities',
       'Louisiana/North Louisiana/Term Life',
       'Louisiana/North Louisiana/Whole Life',
       'Louisiana/Southwest Louisiana/Annuities',
       'Louisiana/Southwest Louisiana/Term Life',


Split the Training and Test data data - Test will be the last 8 quarters

In [9]:
Y_test_df = Y_df.groupby('unique_id').tail(8)
Y_train_df = Y_df.drop(Y_test_df.index)

In [10]:
Y_test_df = Y_test_df.set_index('unique_id')
Y_train_df = Y_train_df.set_index('unique_id')

In [11]:
Y_train_df.groupby('unique_id').size()

unique_id
Louisiana                                   72
Louisiana/Acadiana                          72
Louisiana/Acadiana/Annuities                72
Louisiana/Acadiana/Term Life                72
Louisiana/Acadiana/Whole Life               72
Louisiana/Bayou                             72
Louisiana/Bayou/Annuities                   72
Louisiana/Bayou/Term Life                   72
Louisiana/Bayou/Whole Life                  72
Louisiana/Central Louisiana                 72
Louisiana/Central Louisiana/Annuities       72
Louisiana/Central Louisiana/Term Life       72
Louisiana/Central Louisiana/Whole Life      72
Louisiana/Florida Parishes                  72
Louisiana/Florida Parishes/Annuities        72
Louisiana/Florida Parishes/Term Life        72
Louisiana/Florida Parishes/Whole Life       72
Louisiana/Greater New Orleans               72
Louisiana/Greater New Orleans/Annuities     72
Louisiana/Greater New Orleans/Term Life     72
Louisiana/Greater New Orleans/Whole Life    72
Lou

Compute the base forecasts using AutoETS - this is the forecast for each unique identifier - aka Tag

In [15]:
from statsforecast.models import AutoETS
from statsforecast.core import StatsForecast

In [16]:
fcst = StatsForecast(df=Y_train_df,
                     models=[AutoETS(season_length=4, model='ZZZ')],
                     freq='QS', n_jobs=-1)
Y_hat_df = fcst.forecast(h=8, fitted=True)
Y_fitted_df = fcst.forecast_fitted_values()

C:\Users\K05048\AppData\Local\anaconda3\Lib\site-packages\statsforecast\core.py:399: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
C:\Users\K05048\AppData\Local\anaconda3\Lib\site-packages\statsforecast\core.py:551: FutureWarning: Passing unique_id as the index is deprecated. Please provide it as a column instead.
  warnings.warn(
C:\Users\K05048\AppData\Local\anaconda3\Lib\site-packages\statsforecast\core.py:399: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
C:\Users\K05048\AppData\Local\anaconda3\Lib\site-packages\statsforec

Y_hat_df is the dataframe holding the forecast - of the test dataset

In [96]:
Y_hat_df.head() 

,ds,AutoETS
unique_id,,
Louisiana,2016-01-01,26291.796875
Louisiana,2016-04-01,24458.884766
Louisiana,2016-07-01,23870.267578
Louisiana,2016-10-01,24587.351562
Louisiana,2017-01-01,26291.796875


Y_fitted_df is the dataframe holding the actuals and fitted values of the training dataset.

In [97]:
Y_fitted_df.head() 

,ds,y,AutoETS
unique_id,,,
Louisiana,1998-01-01,23182.230469,22599.873047
Louisiana,1998-04-01,20323.380859,21312.443359
Louisiana,1998-07-01,19826.699219,20286.250000
Louisiana,1998-10-01,20830.130859,20643.957031
Louisiana,1999-01-01,22087.480469,22180.980469


Reconcile the forecasts using a variety of reconcilling algorithms

In [17]:
from hierarchicalforecast.methods import BottomUp, MinTrace, TopDown, MiddleOut
from hierarchicalforecast.core import HierarchicalReconciliation

In [18]:
reconcilers = [
    TopDown(method='forecast_proportions'),
    MiddleOut(middle_level='State/Region',
              top_down_method='forecast_proportions'),
    BottomUp(),
    MinTrace(method='mint_shrink'),
    MinTrace(method='ols')
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_fitted_df, S=S_df, tags=tags)

The dataframe `Y_rec_df` contains the reconciled forecasts on the test data.

In [19]:
Y_rec_df.head(20)

,ds,AutoETS,AutoETS/TopDown_method-forecast_proportions,AutoETS/MiddleOut_middle_level-State/Region_top_down_method-forecast_proportions,AutoETS/BottomUp,AutoETS/MinTrace_method-mint_shrink,AutoETS/MinTrace_method-ols
unique_id,,,,,,,
Louisiana,2016-01-01,26291.796875,26291.796875,25863.179688,25594.462891,25797.988964,26220.968699
Louisiana,2016-04-01,24458.884766,24458.884766,24168.992188,23890.505859,24132.837725,24407.525701
Louisiana,2016-07-01,23870.267578,23870.267578,23595.554688,23304.646484,23608.997587,23820.633369
Louisiana,2016-10-01,24587.351562,24587.351562,24287.082031,24076.539062,24327.654394,24536.936607
Louisiana,2017-01-01,26291.796875,26291.796875,25916.410156,25891.490234,26043.693817,26237.279967
Louisiana,2017-04-01,24458.884766,24458.884766,24222.222656,24149.085938,24358.153739,24422.463819
Louisiana,2017-07-01,23870.267578,23870.267578,23648.783203,23547.701172,23825.969456,23835.017021
Louisiana,2017-10-01,24587.351562,24587.351562,24340.310547,24328.416016,24549.712180,24551.635335
Louisiana/Acadiana,2016-01-01,1715.446045,1743.875244,1715.446045,1691.785400,1705.740636,1762.652009


Evaluate the Forecast using RMSE, MASE, MAPE and MSE metricts

In [20]:
from hierarchicalforecast.evaluation import HierarchicalEvaluation

In [1]:
def rmse(y, y_hat):
    return np.mean(np.sqrt(np.mean((y-y_hat)**2, axis=1)))

def mase(y, y_hat, y_insample, seasonality=4):
    errors = np.mean(np.abs(y - y_hat), axis=1)
    scale = np.mean(np.abs(y_insample[:, seasonality:] - y_insample[:, :-seasonality]), axis=1)
    return np.mean(errors / scale)

def mape(y, y_hat):
    return np.mean((y-y_hat)/y)

def mse(y, y_hat):
    return np.mean((y-y_hat)**2)

eval_tags = {}
eval_tags['State'] = tags['State']
eval_tags['Regions'] = tags['State/Region']
eval_tags['Bottom'] = tags['State/Region/Product']
eval_tags['All'] = np.concatenate(list(tags.values()))

evaluator = HierarchicalEvaluation(evaluators=[rmse, mase,mape,mse])
evaluation = evaluator.evaluate(
        Y_hat_df=Y_rec_df, Y_test_df=Y_test_df,
        tags=eval_tags, Y_df=Y_train_df
)
evaluation = evaluation.drop('Overall')
evaluation.columns = ['Base', 'TopDown','MiddleOut','BottomUp', 'MinTrace(mint_shrink)', 'MinTrace(ols)']
evaluation = evaluation.applymap('{:.2f}'.format)

NameError: name 'tags' is not defined

Root Mean Squared Error

In [59]:
evaluation.query('metric == "rmse"').reset_index()

,level,metric,Base,TopDown,MiddleOut,BottomUp,MinTrace(mint_shrink),MinTrace(ols)
0,State,rmse,1715.47,1715.47,1938.83,2033.64,1831.24,1748.52
1,Regions,rmse,298.40,278.29,298.40,299.40,282.69,282.03
2,Bottom,rmse,118.63,113.06,118.95,118.63,114.15,114.19
3,All,rmse,210.60,201.67,217.60,220.48,207.04,204.40


Mean Absolute Scaled Error

In [60]:
evaluation.query('metric == "mase"').reset_index()

,level,metric,Base,TopDown,MiddleOut,BottomUp,MinTrace(mint_shrink),MinTrace(ols)
0,State,mase,1.53,1.53,1.81,1.98,1.72,1.57
1,Regions,mase,1.31,1.20,1.31,1.39,1.26,1.19
2,Bottom,mase,1.17,1.09,1.13,1.17,1.10,1.08
3,All,mase,1.21,1.13,1.20,1.25,1.16,1.12


Mean Absolute Percentage Error

In [2]:
evaluation.query('metric == "mape"').reset_index()

NameError: name 'evaluation' is not defined

Mean Squared Error

In [62]:
evaluation.query('metric == "mse"').reset_index()

,level,metric,Base,TopDown,MiddleOut,BottomUp,MinTrace(mint_shrink),MinTrace(ols)
0,State,mse,2942847.06,2942847.06,3759076.10,4135704.54,3353444.43,3057306.88
1,Regions,mse,154317.57,131005.87,154317.57,142817.71,134163.96,137908.09
2,Bottom,mse,22182.88,20953.05,23726.26,22182.88,21127.82,21637.37
3,All,mse,142720.51,136174.76,168577.18,176079.86,149509.81,141814.19


### References
- [Forecast Reconcilation with Austrailian Domestic Tourism](https://colab.research.google.com/github/Nixtla/hierarchicalforecast/blob/main/nbs/examples/AustralianDomesticTourism.ipynb)
- [Hyndman, R.J., & Athanasopoulos, G. (2021). "Forecasting: principles and practice, 3rd edition:
Chapter 11: Forecasting hierarchical and grouped series.". OTexts: Melbourne, Australia. OTexts.com/fpp3
Accessed on July 2022.](https://otexts.com/fpp3/hierarchical.html)
- [Rob Hyndman, Alan Lee, Earo Wang, Shanika Wickramasuriya, and Maintainer Earo Wang (2021). "hts: Hierarchical and Grouped Time Series". URL https://CRAN.R-project.org/package=hts. R package version 0.3.1.](https://cran.r-project.org/web/packages/hts/index.html)
- [Mitchell O’Hara-Wild, Rob Hyndman, Earo Wang, Gabriel Caceres, Tim-Gunnar Hensel, and Timothy Hyndman (2021). "fable: Forecasting Models for Tidy Time Series". URL https://CRAN.R-project.org/package=fable. R package version 6.0.2.](https://CRAN.R-project.org/package=fable)